# 3. Diversity

In [2]:
conda activate qiime2-2023.2

(qiime2-2023.2) 


: 1

<img src="../assets/img/qiime_map.svg"  width="1200" height="600">

## 4. Diversity analysis

We want to know:
- the diversity of each sample (alpha diversity)
- and how diverse are each pair of samples (beta diversity)

But before doing these analyses, we need to make all the sequences comparable.

### 4.1 Alignment and tree construction

To do so we are going to:
- align all the sequences
- compute their phylogenetic tree

Thankfully, we can do it in one go with `qiime phylogeny`

In [3]:
qiime phylogeny --help

Usage: qiime phylogeny [OPTIONS] COMMAND [ARGS]...

  Description: This QIIME 2 plugin supports generating and manipulating
  phylogenetic trees.

  Plugin website: https://github.com/qiime2/q2-phylogeny

  Getting user support: Please post to the QIIME 2 forum for help with this
  plugin: https://forum.qiime2.org

Options:
  --version            Show the version and exit.
  --example-data PATH  Write example data and exit.
  --citations          Show citations and exit.
  --help               Show this message and exit.

Commands:
  align-to-tree-mafft-fasttree  Build a phylogenetic tree using fasttree and
                                mafft alignment
  align-to-tree-mafft-iqtree    Build a phylogenetic tree using iqtree and
                                mafft alignment.
  align-to-tree-mafft-raxml     Build a phylogenetic tree using raxml and
                                mafft alignment.
  fasttree                      Construct a phylogenetic tree with FastTree.
  filter-tabl

: 1

There are three methods to align and build the tree:
- MAFFT + FastTree (fastest)  <- this
- MAFFT + IQTREE
- MAFFT + RAxML (most precise)

Let's see the help to know what we need to give it in order to work:

In [4]:
qiime phylogeny align-to-tree-mafft-fasttree --help

Usage: qiime phylogeny align-to-tree-mafft-fasttree [OPTIONS]

  This pipeline will start by creating a sequence alignment using MAFFT, after
  which any alignment columns that are phylogenetically uninformative or
  ambiguously aligned will be removed (masked). The resulting masked alignment
  will be used to infer a phylogenetic tree and then subsequently rooted at
  its midpoint. Output files from each step of the pipeline will be saved.
  This includes both the unmasked and masked MAFFT alignment from q2-alignment
  methods, and both the rooted and unrooted phylogenies from q2-phylogeny
  methods.

Inputs:
  --i-sequences ARTIFACT FeatureData[Sequence]
                          The sequences to be used for creating a fasttree
                          based rooted phylogenetic tree.           [required]
Parameters:
  --p-n-threads VALUE Int % Range(1, None) | Str % Choices('auto')
                          The number of threads. (Use `auto` to automatically
                        

: 1

In [7]:
qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences        rep-seqs.qza \
    --o-alignment        phylo-aligned-seqs.qza \
    --o-masked-alignment phylo-masked-aligned-seqs.qza \
    --o-tree             phylo-unrooted-tree.qza \
    --o-rooted-tree      phylo-rooted-tree.qza

Saved FeatureData[AlignedSequence] to: phylo-aligned-seqs.qza
Saved FeatureData[AlignedSequence] to: phylo-masked-aligned-seqs.qza
Saved Phylogeny[Unrooted] to: phylo-unrooted-tree.qza
Saved Phylogeny[Rooted] to: phylo-rooted-tree.qza
(qiime2-2023.2) 


: 1

All the files are artifacts, so there is nothing to see :(

## 4.2 Core metrics

Also, the alpha and beta diversities are computed in one single command. It is done with `qiime diversity core-metrics-phylogenetic`:

In [8]:
qiime diversity core-metrics-phylogenetic --help

Usage: qiime diversity core-metrics-phylogenetic [OPTIONS]

  Applies a collection of diversity metrics (both phylogenetic and non-
  phylogenetic) to a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table containing the samples over which
                          diversity metrics should be computed.     [required]
  --i-phylogeny ARTIFACT  Phylogenetic tree containing tip identifiers that
    Phylogeny[Rooted]     correspond to the feature identifiers in the table.
                          This tree can contain tip ids that are not present
                          in the table, but all feature ids in the table must
                          be present in this tree.                  [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)        The total frequency that each sample should be
                          rarefied to prior to computing diversity metrics.
                                           

: 1

In [12]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny      phylo-rooted-tree.qza \
  --i-table          table.qza \
  --p-sampling-depth 1200 \
  --m-metadata-file  sample-metadata.tsv \
  --output-dir       metrics

Saved FeatureTable[Frequency] to: metrics/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: metrics/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: metrics/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: metrics/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: metrics/evenness_vector.qza
Saved DistanceMatrix to: metrics/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: metrics/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: metrics/jaccard_distance_matrix.qza
Saved DistanceMatrix to: metrics/bray_curtis_distance_matrix.qza
Saved PCoAResults to: metrics/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: metrics/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: metrics/jaccard_pcoa_results.qza
Saved PCoAResults to: metrics/bray_curtis_pcoa_results.qza
Saved Visualization to: metrics/unweighted_unifrac_emperor.qzv
Saved Visualization to: metrics/weighted_unifrac_emperor.qzv
Saved Visualization to: metrics/jac

: 1

The results appear in the diversity-core-metrics-results folder

In [14]:
ls -1 metrics

bray_curtis_distance_matrix.qza
bray_curtis_emperor.qzv
bray_curtis_pcoa_results.qza
evenness_vector.qza
faith_pd_vector.qza
jaccard_distance_matrix.qza
jaccard_emperor.qzv
jaccard_pcoa_results.qza
observed_features_vector.qza
rarefied_table.qza
shannon_vector.qza
unweighted_unifrac_distance_matrix.qza
unweighted_unifrac_emperor.qzv
unweighted_unifrac_pcoa_results.qza
weighted_unifrac_distance_matrix.qza
weighted_unifrac_emperor.qzv
weighted_unifrac_pcoa_results.qza
(qiime2-2023.2) 


: 1

The visualizable ones:

- [metrics/bray_curtis_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray_curtis_emperor.qzv)
- [metrics/jaccard_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fjaccard_emperor.qzv)
- [metrics/unweighted_unifrac_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted_unifrac_emperor.qzv)
- [metrics/weighted_unifrac_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fweighted_unifrac_emperor.qzv)

## 4.3 Alpha diversity

In [16]:
# Faith
qiime diversity alpha-group-significance \
  --i-alpha-diversity metrics/faith_pd_vector.qza \
  --m-metadata-file   sample-metadata.tsv \
  --o-visualization   metrics/faith-pd-group-significance.qzv

(qiime2-2023.2) 
Saved Visualization to: metrics/faith-pd-group-significance.qzv
(qiime2-2023.2) 


: 1

In [17]:
# evenness
qiime diversity alpha-group-significance \
  --i-alpha-diversity metrics/evenness_vector.qza \
  --m-metadata-file   sample-metadata.tsv \
  --o-visualization   metrics/evenness-group-significance.qzv

(qiime2-2023.2) 
Saved Visualization to: metrics/evenness-group-significance.qzv
(qiime2-2023.2) 


: 1

- [metrics/faith-pd-group-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Ffaith-pd-group-significance.qzv)
- [metrics/evenness-group-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fevenness-group-significance.qzv)

## 4.4 Beta diversity

In [18]:
qiime diversity beta-group-significance \
  --i-distance-matrix metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file   sample-metadata.tsv \
  --m-metadata-column body-site \
  --o-visualization   metrics/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

Saved Visualization to: metrics/unweighted-unifrac-body-site-significance.qzv
(qiime2-2023.2) 


: 1

In [19]:
qiime diversity beta-group-significance \
  --i-distance-matrix metrics/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file   sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization   metrics/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

Saved Visualization to: metrics/unweighted-unifrac-subject-group-significance.qzv
(qiime2-2023.2) 


: 1

- [metrics/unweighted-unifrac-body-site-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-body-site-significance.qzv)
- [metrics/unweighted-unifrac-subject-group-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-subject-group-significance.qzv)

Let's do the emperor plots

In [20]:
qiime emperor plot \
  --i-pcoa          metrics/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes   days-since-experiment-start \
  --o-visualization metrics/unweighted-unifrac-emperor-days-since-experiment-start.qzv

Saved Visualization to: metrics/unweighted-unifrac-emperor-days-since-experiment-start.qzv
(qiime2-2023.2) 


: 1

In [21]:
qiime emperor plot \
  --i-pcoa          metrics/bray_curtis_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes   days-since-experiment-start \
  --o-visualization metrics/bray-curtis-emperor-days-since-experiment-start.qzv

Saved Visualization to: metrics/bray-curtis-emperor-days-since-experiment-start.qzv
(qiime2-2023.2) 


: 1

- [metrics/unweighted-unifrac-emperor-days-since-experiment-start.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray-curtis-emperor-days-since-experiment-start.qzv)
- [metrics/bray-curtis-emperor-days-since-experiment-start.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-emperor-days-since-experiment-start.qzv)

## 4.5 Alpha rarefaction

In [28]:
qiime diversity alpha-rarefaction \
    --i-table         table.qza \
    --i-phylogeny     phylo-rooted-tree.qza \
    --p-max-depth     4000 \
    --m-metadata-file sample-metadata.tsv \
    --o-visualization metrics/alpha-rarefaction.qzv

Saved Visualization to: metrics/alpha-rarefaction.qzv
(qiime2-2023.2) 


: 1

- [alpha_rarefaction.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Falpha-rarefaction.qzv)